# Compound shell

## Import packages

In [1]:
import pandas as pd
import numpy as np
from astropy import constants
from astronavigation.deflection import *
from astronavigation.planets import Body, SolarSystem
import numpy.linalg as LA

np.set_printoptions(precision=54)

## Useful constants

In [2]:
pc = constants.pc.to('km').value
AU = constants.au.to('km').value
c = constants.c.to('km/s').value
eps = 1/c

# Create Solar System
ss = SolarSystem()

jupiter = ss.getPlanet('jupiter')
r_jup_arc = 20  # mean jupiter radius in arcseconds

sun = ss.getSun()

## Read and clean data

Read data

In [3]:
# path
path = 'Stars_GareqEvent2017_oneTransit_new2.dat'

# read, use delimiter='\s+' to choose every white space as delimiter
stars_data = pd.read_table(path, delimiter='\s+')

# display
pd.set_option('display.max_columns', None)
stars_data.head(10)

,tAF5,tAFx,starId,etaS[deg],sig_eta[mas],zetaS[deg],sig_zeta[mas],etaJup[deg],zetaJup[deg],etaSpin[deg],zetaSpin[deg],jupPosGCX[m],jupPosGCY[m],jupPosGCZ[m],gaiaPosX[m],gaiaPosY[m],gaiaPosZ[m],long_jupGC[deg],lat_jupGC[deg],long_gaia[deg],lat_gaia[deg],starRA[deg],starDec[deg]
0,2017.145239,2017.145239,3631075715518049024,4.475043e-01,0.099969,0.149847,0.372775,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.552776,-7.831722
1,2017.145239,2017.145239,3631485528413582336,-5.870432e-01,0.072845,0.052752,0.312333,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.463499,-6.796371
2,2017.145239,2017.145239,3631191099815174272,1.157136e-01,0.091820,0.075482,0.339177,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.480483,-7.499324
3,2017.145239,2017.145239,3631256894418771584,-1.456322e-01,0.096824,-0.074359,0.311625,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.331641,-7.236695
4,2017.145239,2017.145239,3631266072764288000,-2.651681e-01,0.103025,-0.023983,0.389037,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.383427,-7.117590
5,2017.145239,2017.145239,3631298199119733504,-5.711045e-01,0.076116,0.208823,0.297541,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.620517,-6.813578
6,2017.145239,2017.145239,3631241265033227392,-5.069712e-02,0.074388,0.185999,0.254893,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.593303,-7.333806
7,2017.145239,2017.145239,3631239925003381248,-1.514225e-07,0.100975,-0.000004,0.368185,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.405330,-7.382970
8,2017.145239,2017.145239,3631255107712373376,-2.825577e-01,0.065251,0.157291,0.272657,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.566250,-7.101704
9,2017.145239,2017.145239,3631145740665759616,6.684247e-02,0.093621,-0.075513,0.311841,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.328604,-7.449173


Create queary for source_id and add parallaxes to the data

In [4]:
# not need to evaluate this cell if new data has been already downloaded
with open('query.txt', 'w') as f:
    
    star_id = stars_data['starId'].unique()
    query = 'SELECT *\nFROM gaiaedr3.gaia_source\nWHERE '
    
    for id in star_id:
        query += f'source_id = {id} '
        if id != star_id[-1]:
            query += 'OR '
    f.write(query)

In [5]:
# parallax and parallax erro are in mas
new_data = pd.read_csv('1654093971685O-result.csv')
new_data = new_data[['source_id', 'parallax', 'parallax_error']]
new_data = new_data.set_index('source_id')
new_data

,parallax,parallax_error
source_id,,
3631075715518049024,3.409062,0.032620
3631089188831339264,5.459490,0.035637
3631095407943986048,0.243898,0.018328
3631096438736162944,0.535781,0.026916
3631145740665759616,1.217448,0.033738
3631186628753669888,0.400931,0.014865
3631191099815174272,1.352386,0.018157
3631239925003381248,0.472181,0.016357
3631241265033227392,0.758436,0.020345


In [6]:
# add parallax
parallax = []
parallax_err = []
for id in stars_data['starId']:
    p = new_data['parallax']
    err = new_data['parallax_error']
    parallax.append(p.loc[id])
    parallax_err.append(err.loc[id])
stars_data['parallax[mas]'] = parallax
stars_data['parallax_err[mas]'] = parallax_err
stars_data

,tAF5,tAFx,starId,etaS[deg],sig_eta[mas],zetaS[deg],sig_zeta[mas],etaJup[deg],zetaJup[deg],etaSpin[deg],zetaSpin[deg],jupPosGCX[m],jupPosGCY[m],jupPosGCZ[m],gaiaPosX[m],gaiaPosY[m],gaiaPosZ[m],long_jupGC[deg],lat_jupGC[deg],long_gaia[deg],lat_gaia[deg],starRA[deg],starDec[deg],parallax[mas],parallax_err[mas]
0,2017.145239,2017.145239,3631075715518049024,0.447504,0.099969,0.149847,0.372775,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.664430,10.151667,201.552776,-7.831722,3.409062,0.032620
1,2017.145239,2017.145239,3631485528413582336,-0.587043,0.072845,0.052752,0.312333,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.664430,10.151667,201.463499,-6.796371,10.354598,0.184979
2,2017.145239,2017.145239,3631191099815174272,0.115714,0.091820,0.075482,0.339177,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.664430,10.151667,201.480483,-7.499324,1.352386,0.018157
3,2017.145239,2017.145239,3631256894418771584,-0.145632,0.096824,-0.074359,0.311625,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.664430,10.151667,201.331641,-7.236695,6.970485,0.017854
4,2017.145239,2017.145239,3631266072764288000,-0.265168,0.103025,-0.023983,0.389037,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.664430,10.151667,201.383427,-7.117590,1.330410,0.015664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,2017.145239,2017.145240,3631095407943986048,0.228960,0.071818,0.058662,0.318960,0.012963,0.020873,19.823930,22.785597,-6.550931e+11,-2.570747e+11,-9.134985e+10,-1.338483e+11,6.053366e+10,2.630318e+10,201.426269,-7.396111,155.664862,10.151503,201.462561,-7.612432,0.243898,0.018328
138,2017.145239,2017.145240,3631244559272698112,-0.088713,0.049222,-0.037527,0.220813,0.012963,0.020873,19.823930,22.785597,-6.550931e+11,-2.570747e+11,-9.134985e+10,-1.338483e+11,6.053366e+10,2.630318e+10,201.426269,-7.396111,155.664862,10.151503,201.368255,-7.293935,0.488491,0.018010
139,2017.145239,2017.145240,3631244937229820800,-0.079046,0.054070,0.050917,0.226070,0.012963,0.020873,19.823930,22.785597,-6.550931e+11,-2.570747e+11,-9.134985e+10,-1.338483e+11,6.053366e+10,2.630318e+10,201.426269,-7.396111,155.664862,10.151503,201.457351,-7.304351,2.545210,0.030274
140,2017.145239,2017.145240,3631089188831339264,0.347718,0.055859,0.045807,0.233511,0.012963,0.020873,19.823930,22.785597,-6.550931e+11,-2.570747e+11,-9.134985e+10,-1.338483e+11,6.053366e+10,2.630318e+10,201.426269,-7.396111,155.664862,10.151503,201.448593,-7.731080,5.459490,0.035637


Evaluate impact parameter

In [7]:
# star and jupiter coordinates
star = np.array([stars_data['etaS[deg]'], stars_data['zetaS[deg]']])
jup = np.array([stars_data['etaJup[deg]'], stars_data['zetaJup[deg]']])

# impact parameter in units of jupiter radius
stars_data['b_J'] = np.linalg.norm(star - jup, axis=0) * 3600 / r_jup_arc 

Evaluate proper motion

In [8]:
# proper motion is in mas/yr
pm_df = pd.read_csv('1654093971685O-result.csv')
pm_df = pm_df[['source_id', 'pmra', 'pmdec']]
pm_df = pm_df.set_index('source_id')
# add proper motion
pm_ra = []
pm_dec = []
for id in stars_data['starId']:
    ra_dot = pm_df['pmra']
    dec_dot = pm_df['pmdec']
    pm_ra.append(ra_dot.loc[id])
    pm_dec.append(dec_dot.loc[id])
stars_data['pmRA[mas/yr]'] = pm_ra
stars_data['pmDEC[mas/yr]'] = pm_dec
stars_data.head()

,tAF5,tAFx,starId,etaS[deg],sig_eta[mas],zetaS[deg],sig_zeta[mas],etaJup[deg],zetaJup[deg],etaSpin[deg],zetaSpin[deg],jupPosGCX[m],jupPosGCY[m],jupPosGCZ[m],gaiaPosX[m],gaiaPosY[m],gaiaPosZ[m],long_jupGC[deg],lat_jupGC[deg],long_gaia[deg],lat_gaia[deg],starRA[deg],starDec[deg],parallax[mas],parallax_err[mas],b_J,pmRA[mas/yr],pmDEC[mas/yr]
0,2017.145239,2017.145239,3631075715518049024,0.447504,0.099969,0.149847,0.372775,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.552776,-7.831722,3.409062,0.032620,81.587075,-57.337299,-22.363290
1,2017.145239,2017.145239,3631485528413582336,-0.587043,0.072845,0.052752,0.312333,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.463499,-6.796371,10.354598,0.184979,108.155070,-99.783487,9.701552
2,2017.145239,2017.145239,3631191099815174272,0.115714,0.091820,0.075482,0.339177,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.480483,-7.499324,1.352386,0.018157,20.941453,-9.999470,4.156140
3,2017.145239,2017.145239,3631256894418771584,-0.145632,0.096824,-0.074359,0.311625,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.331641,-7.236695,6.970485,0.017854,33.301944,-76.473223,-7.650200
4,2017.145239,2017.145239,3631266072764288000,-0.265168,0.103025,-0.023983,0.389037,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.383427,-7.117590,1.330410,0.015664,50.712957,-5.287082,-0.106522


In [9]:
# shift alpha and delta with proper motion
conv = 1e-3/3600  # from mas to deg

stars_data['starRA[deg]_new'] = stars_data['starRA[deg]'] + stars_data['pmRA[mas/yr]']*(stars_data['tAFx']-2016.0)*conv
stars_data['starDec[deg]_new'] = stars_data['starDec[deg]'] + stars_data['pmDEC[mas/yr]']*(stars_data['tAFx']-2016.0)*conv

Evaluate parallax displacement

In [10]:
# need to transform mas to m^-1
conv = 1e-6/pc  # 1e-3/(pc*1e3)
# useful quantities
alpha = np.deg2rad(stars_data['starRA[deg]_new'])
delta = np.deg2rad(stars_data['starDec[deg]_new'])
X = stars_data['gaiaPosX[m]']
Y = stars_data['gaiaPosY[m]']
Z = stars_data['gaiaPosZ[m]']
omega = stars_data['parallax[mas]']

d_alpha = omega*(1/np.cos(delta))*(X*np.sin(alpha) - Y*np.cos(alpha)) * conv
d_delta = omega*(X*np.cos(alpha)*np.sin(delta) +Y*np.sin(alpha)*np.sin(delta) - Z*np.cos(delta)) * conv

star_gaia_dir = np.array([np.cos(alpha + d_alpha)*np.cos(delta + d_delta), np.sin(alpha + d_alpha)*np.cos(delta + d_delta), np.sin(delta + d_delta)])
star_gaia_dir = star_gaia_dir

In [11]:
# add d_alpha, d_delta
stars_data['dRA[deg]'] = np.rad2deg(d_alpha)
stars_data['dDEC[deg]'] = np.rad2deg(d_delta)

# add star_gaia_dir
stars_data['starGCdirX'] = star_gaia_dir[0]
stars_data['starGCdirY'] = star_gaia_dir[1]
stars_data['starGCdirZ'] = star_gaia_dir[2]

Evaluate theta

In [12]:
# useful values
eta_s = np.deg2rad(stars_data['etaS[deg]'])
zeta_s = np.deg2rad(stars_data['zetaS[deg]'])
eta_J = np.deg2rad(stars_data['etaJup[deg]'])
zeta_J = np.deg2rad(stars_data['zetaJup[deg]'])

# gnomonic coordinates
# cos_psi = np.sin(zeta_J)*np.sin(zeta_s) + np.cos(zeta_J)*np.cos(zeta_s)*np.cos(eta_J - eta_s)
# the previous factor can be neglected because we take the fraction
x_g = np.cos(zeta_J)*np.sin(eta_J - eta_s)
y_g = np.sin(zeta_J)*np.cos(zeta_s) - np.cos(zeta_J)*np.sin(zeta_s)*np.cos(eta_J - eta_s)
theta = np.arctan2(y_g,x_g)
stars_data['theta'] = theta

In [13]:
theta

0     -2.853116
1     -0.053039
2     -2.653213
3      0.540839
4      0.159972
         ...   
137   -2.968396
138    0.521361
139   -0.315617
140   -3.067247
141   -2.705232
Length: 142, dtype: float64

Display data

In [14]:
stars_data.head(10)

,tAF5,tAFx,starId,etaS[deg],sig_eta[mas],zetaS[deg],sig_zeta[mas],etaJup[deg],zetaJup[deg],etaSpin[deg],zetaSpin[deg],jupPosGCX[m],jupPosGCY[m],jupPosGCZ[m],gaiaPosX[m],gaiaPosY[m],gaiaPosZ[m],long_jupGC[deg],lat_jupGC[deg],long_gaia[deg],lat_gaia[deg],starRA[deg],starDec[deg],parallax[mas],parallax_err[mas],b_J,pmRA[mas/yr],pmDEC[mas/yr],starRA[deg]_new,starDec[deg]_new,dRA[deg],dDEC[deg],starGCdirX,starGCdirY,starGCdirZ,theta
0,2017.145239,2017.145239,3631075715518049024,4.475043e-01,0.099969,0.149847,0.372775,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.552776,-7.831722,3.409062,0.032620,81.587075,-57.337299,-22.363290,201.552758,-7.831729,6.739292e-07,-2.531477e-07,-0.921404,-0.363931,-0.136264,-2.853116
1,2017.145239,2017.145239,3631485528413582336,-5.870432e-01,0.072845,0.052752,0.312333,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.463499,-6.796371,10.354598,0.184979,108.155070,-99.783487,9.701552,201.463467,-6.796368,2.039145e-06,-7.352052e-07,-0.924111,-0.363337,-0.118341,-0.053039
2,2017.145239,2017.145239,3631191099815174272,1.157136e-01,0.091820,0.075482,0.339177,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.480483,-7.499324,1.352386,0.018157,20.941453,-9.999470,4.156140,201.480480,-7.499322,2.668142e-07,-9.904283e-08,-0.922583,-0.363052,-0.130514,-2.653213
3,2017.145239,2017.145239,3631256894418771584,-1.456322e-01,0.096824,-0.074359,0.311625,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.331641,-7.236695,6.970485,0.017854,33.301944,-76.473223,-7.650200,201.331617,-7.236697,1.370928e-06,-5.051098e-07,-0.924071,-0.360868,-0.125969,0.540839
4,2017.145239,2017.145239,3631266072764288000,-2.651681e-01,0.103025,-0.023983,0.389037,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.383427,-7.117590,1.330410,0.015664,50.712957,-5.287082,-0.106522,201.383425,-7.117590,2.618223e-07,-9.587169e-08,-0.923986,-0.361798,-0.123906,0.159972
5,2017.145239,2017.145239,3631298199119733504,-5.711045e-01,0.076116,0.208823,0.297541,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.620517,-6.813578,1.593033,0.015341,110.441922,-9.118550,-20.170386,201.620514,-6.813584,3.145644e-07,-1.130959e-07,-0.923079,-0.365855,-0.118639,-0.311278
6,2017.145239,2017.145239,3631241265033227392,-5.069712e-02,0.074388,0.185999,0.254893,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.593303,-7.333806,0.758436,0.020345,31.851915,4.513326,-8.583775,201.593304,-7.333809,1.498627e-07,-5.510800e-08,-0.922213,-0.365005,-0.127650,-1.202731
7,2017.145239,2017.145239,3631239925003381248,-1.514225e-07,0.100975,-0.000004,0.368185,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.405330,-7.382970,0.472181,0.016357,4.427644,-3.083617,-3.564298,201.405329,-7.382971,9.301377e-08,-3.442134e-08,-0.923303,-0.361938,-0.128501,1.015242
8,2017.145239,2017.145239,3631255107712373376,-2.825577e-01,0.065251,0.157291,0.272657,0.012973,0.020895,20.471557,22.787461,-6.550938e+11,-2.570753e+11,-9.135009e+10,-1.338477e+11,6.053463e+10,2.630360e+10,201.426291,-7.396121,155.66443,10.151667,201.566250,-7.101704,3.914919,0.023334,58.587860,22.925695,-17.254796,201.5662

### Find target stars

In [15]:
# threshold
thr = 5

# create threshold column
stars_data['thr'] = stars_data['b_J'] <= thr

# print number of targets
stars_data['thr'].value_counts()

False    133
True       9
Name: thr, dtype: int64

### Create target and reference dataframe

In [16]:
def ref_targ_split(df):
    
    filt = df['thr']

    target_df = df.loc[filt]
    reference_df = df.loc[(~filt)]
    
    return target_df, reference_df

## Evaluate deflection

In [17]:
# define epochs
epochs = stars_data['tAFx'].unique()

In [18]:
# differences
shell=[]
shell_q=[]
dt_new = []

# loop over epochs
for epoch in epochs:
    
    # take only data for the given epoch
    filt = stars_data['tAFx'] == epoch
    stars_epoch = stars_data.loc[filt]
    
    # split into target and reference
    target_df, reference_df = ref_targ_split(stars_epoch)
    
    theta_target = target_df['theta'].to_numpy()
    theta_reference = reference_df['theta'].to_numpy()
    
    bj_target = target_df['b_J'].to_numpy()
    bj_reference = reference_df['b_J'].to_numpy()
    
    
    shell_epoch = np.zeros((len(target_df), len(reference_df)))
    shell_q_epoch = np.zeros((len(target_df), len(reference_df)))
    
    
    # create body
    x_b = np.array([stars_epoch['jupPosGCX[m]'], stars_epoch['jupPosGCY[m]'], stars_epoch['jupPosGCZ[m]']]).T / 1000# in km
    
    # spin = cartesian(stars_epoch['etaSpin[deg]'].iloc[0], stars_epoch['zetaSpin[deg]'].iloc[0])
    
    body = Body(mass=jupiter.mass,
                pos=x_b[0],
                radius=jupiter.radius,
                J2=jupiter.J2,
                s=np.array([0, 0, 1])) 
    
    # sun
    x_sun = -np.array([stars_epoch['gaiaPosX[m]'], stars_epoch['gaiaPosY[m]'], stars_epoch['gaiaPosZ[m]']]).T/1000 # in km
    
    sun = Body(mass=sun.mass,
               pos=x_sun[0],
               radius=sun.radius)
    
    ##################
    #
    # print
    #
    ##################
    print(f'\n---------- epoch: {epoch} ----------\n')
    
    # loop over target
    x_targ = np.array([target_df['starGCdirX'], target_df['starGCdirY'], target_df['starGCdirZ']]).T
    for i in range(len(target_df)):
        
        # direction
        l0 = x_targ[i]
        
        # triplet
        t, n, m = on_triplet(l0, body.pos)
        
        # deflection
        defl_tar = deflection_mod3(l0, body.pos, eps, body.mass)
        dp_eta_tar = defl_tar * np.cos(theta_target[i])
        
        # deflection quadrupole
        dp1_tar, dp2_tar = deflection_mod3(l0, body.pos, eps, body.mass, body.s, body.J2, body.radius)
        dp_q_eta_tar = dp1_tar * np.cos(theta_target[i]) + dp2_tar * np.sin(theta_target[i])
        
        # sun correction
        dl_sun = deflection_mod3(l0, sun.pos, eps, sun.mass)
        
        t_targ_new = t+(defl_tar + dp1_tar)*n + dp2_tar*m
        t_targ_new = t_targ_new/LA.norm(t_targ_new)
        
        
        ##################
        #
        # print
        #
        ##################
        print(f'\nb_J: {bj_target[i]}')
        print(f'\ndefl_tar: {np.rad2deg(defl_tar)*3600*1e6} muas')
        print(f'dp1_tar: {np.rad2deg(dp1_tar)*3600*1e6} muas')
        print(f'dp2_tar: {np.rad2deg(dp2_tar)*3600*1e6} muas')
        print(f'dp_eta_tar: {np.rad2deg(dp_eta_tar)*3600*1e6} muas')
        print(f'dp_q_eta_tar: {np.rad2deg(dp_q_eta_tar)*3600*1e6} muas\n')
        # print(f'dl_sun: {np.rad2deg(dl_sun)*3600*1e6} muas\n')
        
        
        # loop over reference
        x_ref = np.array([reference_df['starGCdirX'], reference_df['starGCdirY'], reference_df['starGCdirZ']]).T
        for j in range(len(reference_df)):
            
            # direction
            l0 = x_ref[j]
            
            # triplet
            t, n, m = on_triplet(l0, body.pos)
        
            # deflection
            defl_ref = deflection_mod3(l0, body.pos, eps, body.mass)
            dp_eta_ref = defl_ref * np.cos(theta_reference[i])
            
            diff = dp_eta_tar - dp_eta_ref
            
            # deflection quadrupole
            dp1_ref, dp2_ref = deflection_mod3(l0, body.pos, eps, body.mass, body.s, body.J2, body.radius)
            dp_q_eta_ref = dp1_ref * np.cos(theta_reference[j]) + dp2_ref * np.sin(theta_reference[j])
            
            diff_q = dp_q_eta_tar - dp_q_eta_ref
            
            shell_epoch[i, j] = np.rad2deg(diff)*3600*1e6
            shell_q_epoch[i, j] = np.rad2deg(diff_q)*3600*1e6
            
            # sun correction
            dl_sun = deflection_mod3(l0, sun.pos, eps, sun.mass)
            
            t_ref_new = t+(defl_ref + dp1_ref)*n + dp2_ref*m
            t_ref_new = t_ref_new/LA.norm(t_ref_new)
            
            dt_new.append(np.rad2deg(np.arccos(np.dot(t_targ_new, t_ref_new)))*3600*1e6)
            
            ##################
            #
            # print
            #
            ##################
            print(f'\nb_J: {bj_reference[j]}')
            print(f'defl_ref: {np.rad2deg(defl_ref)*3600*1e6} muas')
            print(f'dp1_ref: {np.rad2deg(dp1_ref)*3600*1e6} muas')
            print(f'dp2_ref: {np.rad2deg(dp2_ref)*3600*1e6} muas')
            print(f'dp_eta_ref: {np.rad2deg(dp_eta_ref)*3600*1e6} muas')
            print(f'dp_q_eta_ref: {np.rad2deg(dp_q_eta_ref)*3600*1e6} muas\n')
            # print(f'dl_sun: {np.rad2deg(dl_sun)*3600*1e6} muas\n')
            
    
    shell.append(shell_epoch)
    shell_q.append(shell_q_epoch)


---------- epoch: 2017.1452385988 ----------


b_J: 4.427643850555587

defl_tar: 3818.1267893230747 muas
dp1_tar: 1.2482921122735082 muas
dp2_tar: -2.632888422855534 muas
dp_eta_tar: 2013.735307350294 muas
dp_q_eta_tar: -1.5785560974354929 muas


b_J: 81.58707469920164
defl_ref: 207.1945725642207 muas
dp1_ref: -0.0003935378213349119 muas
dp2_ref: -0.0002471025612450895 muas
dp_eta_ref: -198.63294413282023 muas
dp_q_eta_ref: 0.0004475749590271111 muas


b_J: 108.15506981721754
defl_ref: 156.29905966767274 muas
dp1_ref: -0.00019890231804004096 muas
dp2_ref: 2.454735694175988e-05 muas
dp_eta_ref: -149.84051948251835 muas
dp_q_eta_ref: -0.00019992397350955418 muas


b_J: 20.941452965187587
defl_ref: 807.1988827712722 muas
dp1_ref: -0.015778194978545008 muas
dp2_ref: -0.022545809600284097 muas
dp_eta_ref: -773.8440664795128 muas
dp_q_eta_ref: 0.024512027028917242 muas


b_J: 33.30194388495801
defl_ref: 507.60528378643096 muas
dp1_ref: -0.003323173349347066 muas
dp2_ref: -0.0059914788703127

In [19]:
#shell

In [20]:
#shell_q

Create data

In [123]:
# angular separation
data_y = []
variance =[]
for epoch in epochs:
    
    print(f'\n---------- epoch: {epoch} ----------\n')
    
    # take only data for the given epoch
    filt = stars_data['tAFx'] == epoch
    stars_epoch = stars_data.loc[filt]
    
    # split into target and reference
    target_df, reference_df = ref_targ_split(stars_epoch)
    
    theta_target = target_df['theta'].to_numpy()
    theta_reference = reference_df['theta'].to_numpy()
    
    r_p = np.zeros((len(target_df), len(reference_df)))
    r_m = np.zeros((len(target_df), len(reference_df)))
    
    var = np.zeros((len(target_df), len(reference_df)))
    err_r_p = np.zeros((len(target_df), len(reference_df)))
    err_r_m = np.zeros((len(target_df), len(reference_df)))
    
    eta_targ = target_df['etaS[deg]'].to_numpy()
    eta_ref = reference_df['etaS[deg]'].to_numpy()
    
    zeta_targ = target_df['zetaS[deg]'].to_numpy()
    zeta_ref = reference_df['zetaS[deg]'].to_numpy()
    
    err_eta_targ = target_df['sig_eta[mas]'].to_numpy() * 1e3
    err_eta_ref = reference_df['sig_eta[mas]'].to_numpy() * 1e3
    
    err_zeta_targ = target_df['sig_zeta[mas]'].to_numpy() * 1e3
    err_zeta_ref = reference_df['sig_zeta[mas]'].to_numpy() * 1e3
    
    x_targ = target_df['starGCdirX'].to_numpy()
    y_targ = target_df['starGCdirY'].to_numpy()
    z_targ = target_df['starGCdirZ'].to_numpy()
    
    x_ref = reference_df['starGCdirX'].to_numpy()
    y_ref = reference_df['starGCdirY'].to_numpy()
    z_ref = reference_df['starGCdirZ'].to_numpy()
    
    alpha_targ = target_df['starRA[deg]_new'].to_numpy() + target_df['dRA[deg]'].to_numpy()
    delta_targ = target_df['starDec[deg]_new'].to_numpy() + target_df['dDEC[deg]'].to_numpy()
    alpha_ref =  reference_df['starRA[deg]_new'].to_numpy() +  reference_df['dRA[deg]'].to_numpy()
    delta_ref = reference_df['starDec[deg]_new'].to_numpy() +  reference_df['dDEC[deg]'].to_numpy()
    
    jup_epoch = np.array([stars_epoch['jupPosGCX[m]'], stars_epoch['jupPosGCY[m]'], stars_epoch['jupPosGCZ[m]']]).T
    jup_epoch = jup_epoch[0]
    jup_epoch = jup_epoch/LA.norm(jup_epoch)
    
    alpha_jup = stars_epoch['long_jupGC[deg]'].to_numpy()[0]
    delta_jup = stars_epoch['lat_jupGC[deg]'].to_numpy()[0]
    
    eta_J_tar = target_df['etaJup[deg]'].to_numpy()
    zeta_J_tar = target_df['zetaJup[deg]'].to_numpy()
    
    eta_J_ref = reference_df['etaJup[deg]'].to_numpy()
    zeta_J_ref = reference_df['zetaJup[deg]'].to_numpy()
    
    # loop over target
    for i in range(len(target_df)):
        
        r_targ = np.array([x_targ[i], y_targ[i], z_targ[i]]) 
        
        chi_targ = np.arccos(np.dot(r_targ, jup_epoch))
        chi_targ_eta = chi_targ*np.cos(theta_target[i])
        #print(f'chi_targ:       {chi_targ}')
        #print(f'chi_targ_eta:   {chi_targ_eta}')
        
        eta_rel_targ = eta_targ[i] - eta_J_tar[i]
        zeta_rel_targ = zeta_targ[i] - zeta_J_tar[i]
        
        alpha_rel_targ = alpha_targ[i] - alpha_jup
        delta_rel_targ = delta_targ[i] - delta_jup
        
        
        # a_targ_new = np.deg2rad(np.sqrt(eta_rel_targ**2 + zeta_rel_targ**2))
        a_targ_new = np.arccos(np.dot(cartesian(eta_targ[i], zeta_targ[i]), cartesian(eta_J_tar[i], zeta_J_tar[i])))
        #a_targ = np.deg2rad(np.sqrt(alpha_rel_targ**2 + delta_rel_targ**2))
        a_targ = chi_targ
        print(np.rad2deg(a_targ_new)*3600e6 - np.rad2deg(a_targ)*3600e6)
        
        
        # loop over reference
        for j in range(len(reference_df)):
            
            r_ref = np.array([x_ref[j], y_ref[j], z_ref[j]])
            
            theta_rel = np.arctan2((zeta_ref[j] - zeta_targ[i]) ,  (eta_ref[j] - eta_targ[i]))

            
            chi_ref = np.arccos(np.dot(r_ref, jup_epoch))
            chi_ref_eta = chi_ref*np.cos(theta_reference[j])
            
            #print(f'chi_ref:      {chi_ref}')
            #print(f'chi_ref_eta:  {chi_ref_eta}')
            

            #r_p[i, j] = np.sqrt((eta_targ[i] - eta_ref[j])**2 + (zeta_targ[i] - zeta_ref[j])**2)*3600*1e6
            #r_p[i, j] = np.abs((eta_targ[i] - eta_ref[j])) *3600*1e6
            
            
            r_p[i, j] = rad2muas(np.arccos(np.dot(cartesian(eta_targ[i], zeta_targ[i]), cartesian(eta_ref[j], zeta_ref[j]))))
            r_m[i, j] = rad2muas(np.arccos(np.dot(r_targ, r_ref))) #* np.cos(theta_rel)
            
            # r_m [i, j] = np.sqrt((alpha_targ[i] - alpha_ref[j])**2 + (delta_targ[i] - delta_ref[j])**2)*3600*1e6
            #r_m[i, j] = chi_ref_eta + chi_targ_eta
            
            
            eta_rel_ref = eta_ref[j] - eta_J_ref[j]
            zeta_rel_ref = zeta_ref[j] - zeta_J_ref[j]
        
            alpha_rel_ref = alpha_ref[j] - alpha_jup
            delta_rel_ref = delta_ref[j] - delta_jup
        
            #a_ref_new = np.deg2rad(np.sqrt(eta_rel_ref**2 + zeta_rel_ref**2))
            a_ref_new = np.arccos(np.dot(cartesian(eta_ref[j], zeta_ref[j]), cartesian(eta_J_ref[j], zeta_J_ref[j])))
            #a_ref = np.deg2rad(np.sqrt(alpha_rel_ref**2 + delta_rel_ref**2))
            a_ref = chi_ref
            print(np.rad2deg(a_ref_new)*3600e6 - np.rad2deg(a_ref)*3600e6)
            
            
            #r_p[i, j] = np.rad2deg((np.cos(a_targ_new) - np.cos(a_targ))/np.sin(a_targ))*3600e6
            #r_m[i, j] = np.rad2deg((np.cos(a_ref_new) - np.cos(a_ref))/np.sin(a_ref))*3600e6
            
            
            
            err_r_p[i, j] = LA.norm([err_eta_targ[i], err_eta_ref[j]])
            
            var[i, j] = 2*err_eta_targ[i]**2 + err_eta_ref[j]**2
            
            print(f'sep_j: {r_p[i, j]} muas')
            print(f'sep:   {r_m[i, j]} muas\n')
            # print(f'err: {err_r_p[i, j]} muas\n')
            
                
    data_y.append(np.abs(r_p - r_m ))
    variance.append(var)



---------- epoch: 2017.1452385988 ----------

-369.8095567971468
-87804.44500303268
sep_j: 1698937361.5648944 muas
sep:   1699025064.2708395 muas

-88214.1152100563
sep_j: 2121871202.5462296 muas
sep:   2121958978.7090383 muas

-36131.45287781954
sep_j: 497370222.5763862 muas
sep:   497407394.764641 muas

-44198.2729203701
sep_j: 588656613.5339638 muas
sep:   588700469.8679949 muas

-69898.57918083668
sep_j: 958500091.284178 muas
sep:   958569208.0440698 muas

-95349.23137712479
sep_j: 2189105287.5056744 muas
sep:   2189199766.8189626 muas

-48555.793892383575
sep_j: 694034943.6181889 muas
sep:   694083233.4994341 muas

-80834.79198670387
sep_j: 1164199740.762505 muas
sep:   1164280074.702138 muas

-28478.346463620663
sep_j: 363040442.2334474 muas
sep:   363068933.82586277 muas

-62418.28912162781
sep_j: 936879861.9953325 muas
sep:   936942731.8772132 muas

-58331.58797669411
sep_j: 850879654.7142038 muas
sep:   850939047.2805923 muas

-28357.29459810257
sep_j: 346763651.87507296 muas

In [43]:
data_y

[array([[87702.70594501495 , 87776.16280865669 , 37172.18825477362 ,
         43856.33403110504 , 69116.75989174843 , 94479.31328821182 ,
         48289.88124525547 , 80333.93963289261 , 28491.5924153924  ,
         62869.88188076019 , 59392.566388487816, 28067.114919364452,
         27699.16510552168 , 79044.44870114326 , 41179.74205267429 ]]),
 array([[90673.9598031044  , 86493.68043589592 , 35542.91498887539 ,
         42944.42214536667 , 68542.61462271214 , 90441.80044937134 ,
         47067.993972182274, 78012.81495904922 , 26997.46836209297 ,
         57171.92704319954 , 54198.08866691589 , 28423.55069589615 ,
         24383.58119338751 , 79308.54590988159 , 41598.795741677284]]),
 array([[90284.05870485306 , 86761.51849365234 , 28589.852193832397,
         44107.55418229103 , 68801.98452150822 , 90046.07554197311 ,
         46347.246461987495, 77617.78250932693 , 27783.893871307373,
         52437.37112176418 , 53631.478122234344, 29600.498325288296,
         24120.279978454113,

## Non linear fit

In [27]:
from scipy.optimize import curve_fit

In [28]:
def model(x_data, gamma, epsilon):
    
    dpsi_mono = x_data[0]
    dpsi_quad = x_data[1]
    
    dr = (1+gamma)*(dpsi_mono + epsilon*dpsi_quad)
    
    return dr

In [29]:
shell = [ep.squeeze() for ep in shell]
shell = np.concatenate(shell)

shell_q = [ep.squeeze() for ep in shell_q]
shell_q = np.concatenate(shell_q)

data_y = [ep.squeeze() for ep in data_y]
data_y = np.concatenate(data_y)

In [30]:
xdata = np.array([shell/2, shell_q/2]).squeeze()
ydata = np.array(data_y).squeeze()
ydata

array([87702.70594501495 , 87776.16280841827 , 37172.18825483322 ,
       43856.33403098583 , 69116.75989174843 , 94479.31328821182 ,
       48289.88124525547 , 80333.93963313103 , 28491.5924153924  ,
       62869.88188076019 , 59392.56638836861 , 28067.114919364452,
       27699.16510552168 , 79044.44870114326 , 41179.74205267429 ,
       90673.95980334282 , 86493.68043589592 , 35542.914988934994,
       42944.42214536667 , 68542.61462259293 , 90441.80044937134 ,
       47067.993972182274, 78012.81495928764 , 26997.46836209297 ,
       57171.92704319954 , 54198.08866691589 , 28423.55069589615 ,
       24383.58119338751 , 79308.54590988159 , 41598.79574179649 ,
       90284.05870485306 , 86761.51849389076 , 28589.852193892002,
       44107.55418241024 , 68801.98452138901 , 90046.07554197311 ,
       46347.246462106705, 77617.78250932693 , 27783.893871307373,
       52437.37112188339 , 53631.478122234344, 29600.49832522869 ,
       24120.279978454113, 79352.29026532173 , 32028.907665967

## Monte Carlo

In [83]:
# array for gamma and epsilon
gamma = []
epsilon = []


# loop 
for i in range(10000):
    
    rng = np.random.default_rng()
    y_noise = rng.normal(scale=1, size=xdata.shape[1])
    ydata = np.array(shell) + np.array(shell_q) +y_noise
    ydata = ydata.squeeze()
    
    # constrain optimization
    popt, pcov = curve_fit(model, xdata, ydata, p0=(1,1), bounds=(0, [10, 10]))
    
    gamma.append(popt[0])
    epsilon.append(popt[1])
    
# gamma statistic
mean_gamma = np.mean(gamma)
var_gamma = np.var(gamma)

# epsilon statistic
mean_epsilon = np.mean(epsilon)
var_epsilon = np.var(epsilon)

In [84]:
mean_gamma

1.000009667797697

In [85]:
np.sqrt(var_gamma)

0.0006597876945668349

In [86]:
mean_epsilon

1.0073806889047987

In [87]:
var_epsilon

0.27218142264853634

## Covariance

In [174]:
rng = np.random.default_rng()
y_noise = rng.normal(size=xdata.shape[1])
ydata = np.array(shell) + np.array(shell_q) +y_noise
ydata = ydata.squeeze()

In [125]:
variance = [ep.squeeze() for ep in variance]
variance = np.concatenate(variance)

In [169]:
sigma = np.zeros((len(ydata), len(ydata)))

for i in range(len(ydata)):
    for j in range(len(ydata)):
        
        sigma[i, j] = 1 if i==j else 0

In [170]:
sigma

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [175]:
# constrain optimization
popt, pcov = curve_fit(model, xdata, ydata, sigma=sigma, p0=(1,1), bounds=(0, [10, 10]))

In [176]:
popt

array([0.999410926668243 , 0.5000407996742209])

In [177]:
perr = np.sqrt(np.diag(pcov))
perr

array([0.0006993302808497547, 0.5540163625916563   ])

array([100.9750102539684 ,  85.289853588708  ,  74.0978255734469 ,
        68.72095887140429,  63.4565936178091 ,  66.07312703104961,
        68.4639989220111 ,  66.1654717984665 ,  66.3743315127661 ])

In [150]:
# need to properly write the covariance matrix

# covariance terms
target, _ = ref_targ_split(stars_data)
possible_cov = target['sig_eta[mas]'].to_numpy()*1e3

aa = stars_data['tAFx'].value_counts()

In [159]:
bb = np.cumsum(aa)
bb = bb.to_list()
bb.insert(0, 0)
bb

[0, 16, 32, 48, 64, 80, 96, 112, 127, 142]

In [161]:
epoch_ranges = [(bb[i], bb[i+1]-1) for i in range(len(bb)-1)]

In [162]:
epoch_ranges

[(0, 15),
 (16, 31),
 (32, 47),
 (48, 63),
 (64, 79),
 (80, 95),
 (96, 111),
 (112, 126),
 (127, 141)]

In [ ]:
sigma = np.zeros((len(ydata), len(ydata)))

for i in range(len(ydata)):
    for j in range(len(ydata)):
        
        for rng in epoch_ranges:
            
        
        sigma[i, j] = variance[i] if i==j else cov

In [178]:
for rng in epoch_ranges:
    
    print(range(rng[0], rng[1]))

range(0, 15)
range(16, 31)
range(32, 47)
range(48, 63)
range(64, 79)
range(80, 95)
range(96, 111)
range(112, 126)
range(127, 141)


In [165]:
stars_data['tAFx'].unique()

array([2017.1452385988   , 2017.145238752673 , 2017.1452389066772,
       2017.1452390606796, 2017.145239214684 , 2017.1452393686895,
       2017.145239522684 , 2017.1452396766856, 2017.1452398305944])